In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import os
from PIL import Image
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import cv2 
import math
#
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
from keras.utils import multi_gpu_model
#
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.preprocessing import image
import warnings
from os.path import split

In [6]:
# Reference: https://www.kaggle.com/gimunu/data-augmentation-with-keras-into-cnn
# Load images
train_images = os.listdir("/media/fanglab/DATA/Wei-Tse/Whale_Classifier/train/")
test_images = os.listdir("/media/fanglab/DATA/Wei-Tse/Whale_Classifier/test/")
df = pd.read_csv("train.csv")
# Build Table
df["imagepath"] = df["Image"].map( lambda x : "/media/fanglab/DATA/Wei-Tse/Whale_Classifier/train/"+x)
ImageToLabelDict = dict( zip( df.Image, df["Id"]))
df.head()

,Image,Id,imagepath
0,00022e1a.jpg,w_e15442c,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
1,000466c4.jpg,w_1287fbc,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
2,00087b01.jpg,w_da2efe0,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
3,001296d5.jpg,w_19e5482,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...
4,0014cfdf.jpg,w_f22f3e3,/media/fanglab/DATA/Wei-Tse/Whale_Classifier/t...


In [9]:
import shutil

In [33]:
for category in os.listdir('CNN_TransferL/train/'):
    if not os.path.isdir('CNN_TransferL/dev/'+category):
        os.mkdir('CNN_TransferL/dev/'+category)

In [13]:
# After I submitted to Kaggle,I found my model has high varaince
# So I retrained the model 
# At this time, I split the data into train and dev set
#I only split the data that have more than 20 images
for category in os.listdir('CNN_TransferL/train/'):
    file=os.listdir('CNN_TransferL/train/'+category)
    if len(file)<20:
        continue
    elif len(file)<35:
        if not os.path.isdir('CNN_TransferL/dev/'+category):
            os.mkdir('CNN_TransferL/dev/'+category)
        shutil.move('CNN_TransferL/train/'+category+'/'+file[0],'CNN_TransferL/dev/'+category+'/'+file[0])
    elif len(file)<=810:
        if not os.path.isdir('CNN_TransferL/dev/'+category):
            os.mkdir('CNN_TransferL/dev/'+category)
        for i in range(0,93):
            shutil.move('CNN_TransferL/train/'+category+'/'+file[i],'CNN_TransferL/dev/'+category+'/'+file[i])

In [35]:
# Build CNN imagedatabase

batch_size = 64
seednum = 1

#generator for training
# the folder name will be though as label name if class_mode="categorical"
# And keras will use folder name for one hot encoding
# let's why alphabery order is important


# Data Augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

#data from specific path
trainD = train_gen.flow_from_directory(
        'CNN_TransferL/train/',
        class_mode="categorical",
        batch_size=batch_size,
        target_size=(224, 224),
        seed=seednum,
        shuffle=False)

# Data Augmentation
dev_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

#data from specific path
devD = dev_gen.flow_from_directory(
        'CNN_TransferL/dev/',
        class_mode="categorical",
        batch_size=batch_size,
        target_size=(224, 224),
        seed=seednum,
        shuffle=False)

Found 9745 images belonging to 4251 classes.
Found 105 images belonging to 4251 classes.


In [15]:
# Got One-hot encoding in keras
encoded_dict=trainD.class_indices
# Inverse encoded : labelname
dict_encodedlabel={a :b for b,a in encoded_dict.items()}

In [17]:
# LabelEncoder + OnehotEncoder
class LabelOneHotEncoder():
    def __init__(self):
        self.ohe = OneHotEncoder()
        self.le = LabelEncoder()
    def fit_transform(self, x):
        features = self.le.fit_transform( x)
        return self.ohe.fit_transform( features.reshape(-1,1))
    def transform( self, x):
        return self.ohe.transform( self.la.transform( x.reshape(-1,1)))
    def inverse_tranform( self, x):
        return self.le.inverse_transform( self.ohe.inverse_tranform( x))
    def inverse_labels( self, x):
        return self.le.inverse_transform( x)
#constructing class weights
WeightFunction = lambda x : 1./x**0.75
ClassLabel2Index = lambda x : lohe.le.inverse_tranform([[x]])

In [84]:
# Keras use alphabery order for one-hot enciding
# Sklearn also use alphabery order for one-hot enciding
# So, I guess my weighting order can match the order in imageDataGenrator.flow_from_dictionary
lohe = LabelOneHotEncoder()
# find the label name
y = list(map(ImageToLabelDict.get, train_images))
# Encode label
y_cat = lohe.fit_transform(y)
#
CountDict = dict( df["Id"].value_counts())
# set the weighting dictionary 
class_weight_dic = {lohe.le.transform( [image_name])[0]:WeightFunction(count) for image_name, count in CountDict.items()}

In [38]:
# Train new model with Resnet50
from keras.applications.resnet50 import ResNet50
# Import Resnet 
ResNet50model = ResNet50(weights='imagenet', include_top=True)

In [72]:
# Extract features from flatten_6 layer
# Add to new FC layer 
x = Dense(4096, activation='relu')(ResNet50model.get_layer('flatten_2').output)
# Final Output Layer 4251 is # of categories
predictions = Dense(4251, activation='softmax')(x)
# Connect all layers
ResNet50model_TL=Model(inputs=ResNet50model.input, outputs=predictions)

In [73]:
# transfer learning only update FC layers
for layerG in ResNet50model_TL.layers[:176]:
   layerG.trainable = False

In [74]:
# optimizer 
ot=keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.95, amsgrad=False)
#Set model trained with 2 gpus 
ResNet50model_TL = multi_gpu_model(ResNet50model_TL, gpus=2)
#Compile the model
ResNet50model_TL.compile(optimizer=ot, loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

In [ ]:
# Train
epochsnum = 20
ResNet50model_TL.fit_generator(trainD,
          steps_per_epoch=9745//batch_size,
          epochs=epochsnum,
          verbose=1,
          class_weight=class_weight_dic,
          shuffle=False,
          validation_steps=105//batch_size,
          validation_data=devD, callbacks=callbacks_list)

In [86]:
# Import VGG19
from keras.applications.vgg19 import VGG19
vggorigin = VGG19(weights='imagenet', include_top=True)
predictions = Dense(4251, activation='softmax')(vggorigin.get_layer('fc2').output)
VGG_TL=Model(inputs=vggorigin.input, outputs=predictions)
# transfer learning only update FC layers
for layerG in VGG_TL.layers[:24]:
   layerG.trainable = False

In [87]:
# optimizer 
ot=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.95, amsgrad=False)
#Set model trained with 2 gpus 
VGG_TL = multi_gpu_model(VGG_TL, gpus=2)
#Compile the model
VGG_TL.compile(optimizer=ot, loss='categorical_crossentropy', metrics=['accuracy'])

In [88]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

In [ ]:
# Train
epochsnum = 20
VGG_TL.fit_generator(trainD,
          steps_per_epoch=9745//batch_size,
          epochs=epochsnum,
          verbose=1,
          class_weight=class_weight_dic,
          shuffle=False,
          validation_steps=105//batch_size,
          validation_data=devD, callbacks=callbacks_list)

Epoch 1/20
139/152 [==========================>...] - ETA: 10s - loss: 8.0203 - acc: 0.0733